In [1]:
# Set autoreload
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
sys.path.append('../app/src')
from config import config
import engines
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from engines import setup


In [2]:
messages_history = ChatMessageHistory()
db = SQLDatabase.from_uri('sqlite:///../data/artifacts/sql_database.db')

agent = setup.create_sql_database_agent(
    messages_history=messages_history, 
    db=db
)


In [3]:
answer = agent.get_answer("Get the final highest earning for clients whose Analyst Rating was to sell the shares")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


client_target_allocations, financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'financial_advisor_clients'}`



CREATE TABLE financial_advisor_clients (
	"Client" TEXT, 
	"Symbol" TEXT, 
	"Name" TEXT, 
	"Sector" TEXT, 
	"Quantity" REAL, 
	"Buy Price" REAL, 
	"Current Price" REAL, 
	"Market Value" REAL, 
	"Purchase Date" TEXT, 
	"Dividend Yield" REAL, 
	"P/E Ratio" REAL, 
	"52-Week High" REAL, 
	"52-Week Low" REAL, 
	"Analyst Rating" TEXT, 
	"Target Price" REAL, 
	"Risk Level" TEXT, 
	"Client_validation" INTEGER, 
	"Client_Id" TEXT
)

/*
3 rows from financial_advisor_clients table:
Client	Symbol	Name	Sector	Quantity	Buy Price	Current Price	Market Value	Purchase Date	Dividend Yield	P/E Ratio	52-Week High	52-Week Low	Analyst Rating	Target Price	Risk Level	Client_validation	Client_Id
Client_1	DIA	SPDR Dow Jones Industrial Average ETF	ETF	135.0	264.89	574.42	77546.7	12/11/21	1.61	1

In [4]:
print(answer.get("output"))

The highest final earning for clients whose Analyst Rating was to sell the shares is $260,663. This earning belongs to Client_13.


In [5]:
answer = agent.get_answer("Which client holds stocks from Google")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


client_target_allocations, financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'financial_advisor_clients'}`



CREATE TABLE financial_advisor_clients (
	"Client" TEXT, 
	"Symbol" TEXT, 
	"Name" TEXT, 
	"Sector" TEXT, 
	"Quantity" REAL, 
	"Buy Price" REAL, 
	"Current Price" REAL, 
	"Market Value" REAL, 
	"Purchase Date" TEXT, 
	"Dividend Yield" REAL, 
	"P/E Ratio" REAL, 
	"52-Week High" REAL, 
	"52-Week Low" REAL, 
	"Analyst Rating" TEXT, 
	"Target Price" REAL, 
	"Risk Level" TEXT, 
	"Client_validation" INTEGER, 
	"Client_Id" TEXT
)

/*
3 rows from financial_advisor_clients table:
Client	Symbol	Name	Sector	Quantity	Buy Price	Current Price	Market Value	Purchase Date	Dividend Yield	P/E Ratio	52-Week High	52-Week Low	Analyst Rating	Target Price	Risk Level	Client_validation	Client_Id
Client_1	DIA	SPDR Dow Jones Industrial Average ETF	ETF	135.0	264.89	574.42	77546.7	12/11/21	1.61	1

In [6]:
print(answer.get("output"))

The following clients hold stocks from Google (Alphabet Inc.):

1. Client_1
2. Client_2
3. Client_3
4. Client_4
5. Client_5
6. Client_6
7. Client_7
8. Client_8
9. Client_9
10. Client_10


In [7]:
answer = agent.get_answer("and which client holds 10% or more in cash")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


client_target_allocations, financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'client_target_allocations'}`



CREATE TABLE client_target_allocations (
	"Client" TEXT, 
	"Target Portfolio" TEXT, 
	"Asset Class" TEXT, 
	"Target Allocation (%)" REAL, 
	"Target Portfolio_validation" INTEGER, 
	"Client_validation" INTEGER, 
	"Asset Class_validation" INTEGER
)

/*
3 rows from client_target_allocations table:
Client	Target Portfolio	Asset Class	Target Allocation (%)	Target Portfolio_validation	Client_validation	Asset Class_validation
Client_1	Balanced	Bonds	30.0	1	1	1
Client_1	Balanced	ETFs	15.0	1	1	1
Client_1	Balanced	Cash	5.0	1	1	1
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT Client, "Target Portfolio", "Target Allocation (%)" FROM client_target_allocations WHERE "Asset Class" = \'Cash\' AND "Target Allocation (%)" >= 10 LIMIT 10'}`


```sql
SELECT Client, "Target 

In [8]:
print(answer.get("output"))

The clients who hold 10% or more in cash are:

1. Client_7 (Conservative) - 10.0%
2. Client_8 (Conservative) - 10.0%
3. Client_21 (Conservative) - 10.0%
4. Client_22 (Conservative) - 10.0%
5. Client_30 (Conservative) - 10.0%
6. Client_33 (Conservative) - 10.0%
7. Client_41 (Conservative) - 10.0%
8. Client_48 (Conservative) - 10.0%


In [9]:
answer = agent.get_answer("what about 5%?")




> Entering new SQL Agent Executor chain...
I don't know. Could you please provide more context or clarify your question? Are you asking about a specific percentage related to a financial metric, a discount, or something else?

> Finished chain.


In [10]:
print(answer.get("output"))

I don't know. Could you please provide more context or clarify your question? Are you asking about a specific percentage related to a financial metric, a discount, or something else?


In [11]:
answer = agent.get_answer("I wanna know which client holds more than 5% in cash")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


client_target_allocations, financial_advisor_clients
Invoking: `sql_db_schema` with `{'table_names': 'client_target_allocations'}`



CREATE TABLE client_target_allocations (
	"Client" TEXT, 
	"Target Portfolio" TEXT, 
	"Asset Class" TEXT, 
	"Target Allocation (%)" REAL, 
	"Target Portfolio_validation" INTEGER, 
	"Client_validation" INTEGER, 
	"Asset Class_validation" INTEGER
)

/*
3 rows from client_target_allocations table:
Client	Target Portfolio	Asset Class	Target Allocation (%)	Target Portfolio_validation	Client_validation	Asset Class_validation
Client_1	Balanced	Bonds	30.0	1	1	1
Client_1	Balanced	ETFs	15.0	1	1	1
Client_1	Balanced	Cash	5.0	1	1	1
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT Client, "Target Portfolio", "Target Allocation (%)" FROM client_target_allocations WHERE "Asset Class" = \'Cash\' AND "Target Allocation (%)" > 5.0 LIMIT 10'}`


```sql
SELECT Client, "Target 

In [12]:
print(answer.get("output"))

Here are the clients who hold more than 5% in cash:

1. Client_7 - Target Portfolio: Conservative, Cash Allocation: 10.0%
2. Client_8 - Target Portfolio: Conservative, Cash Allocation: 10.0%
3. Client_21 - Target Portfolio: Conservative, Cash Allocation: 10.0%
4. Client_22 - Target Portfolio: Conservative, Cash Allocation: 10.0%
5. Client_30 - Target Portfolio: Conservative, Cash Allocation: 10.0%
6. Client_33 - Target Portfolio: Conservative, Cash Allocation: 10.0%
7. Client_41 - Target Portfolio: Conservative, Cash Allocation: 10.0%
8. Client_48 - Target Portfolio: Conservative, Cash Allocation: 10.0%
